In [1]:
import argparse
import pathlib
import random
import warnings

import numpy as np
import pandas as pd
from copairs import map
from copairs.matching import assign_reference_index

# suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
well_to_dose_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve(strict=True)
well_to_dose_df = pd.read_parquet(
    well_to_dose_path, columns=["Metadata_Well", "Metadata_dose"]
)
# drop duplicates
well_to_dose_df = well_to_dose_df.drop_duplicates(
    subset=["Metadata_Well", "Metadata_dose"]
)
# make dict that maps well to dose
well_to_dose = well_to_dose_df.set_index("Metadata_Well")["Metadata_dose"].to_dict()

In [3]:
CP_scDINO_profile_file_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs_aggregated.parquet"
).resolve(strict=True)
df = pd.read_parquet(CP_scDINO_profile_file_path)
df.head()

,Metadata_Well,Metadata_Time,Metadata_dose,Metadata_number_of_singlecells,Metadata_plate,Metadata_compound,Metadata_control,Cells_AreaShape_Compactness_CP,Cells_AreaShape_Eccentricity_CP,Cells_AreaShape_Extent_CP,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,C-02,0.0,0.0,178.0,1,Staurosporine,negative,-0.225866,0.362134,0.195628,...,0.123994,-0.169231,-0.000171,-0.000714,0.219109,0.060974,0.073445,-0.099431,-0.202662,0.069599
1,C-02,1.0,0.0,181.0,1,Staurosporine,negative,-0.101367,0.212559,0.160123,...,-0.110603,0.076735,0.057752,0.065120,-0.253076,0.088509,-0.046158,0.118233,-0.090497,-0.116096
2,C-02,10.0,0.0,168.0,1,Staurosporine,negative,0.018758,0.438709,0.017725,...,-0.077459,-0.076050,0.179852,0.079523,-0.534580,0.115206,0.033339,0.179140,-0.031637,-0.142134
3,C-02,11.0,0.0,174.0,1,Staurosporine,negative,-0.041979,0.460182,-0.009282,...,-0.121237,-0.075781,0.197840,0.229394,-0.400539,0.074929,-0.064636,0.225648,-0.063553,-0.139911
4,C-02,12.0,0.0,175.0,1,Staurosporine,negative,0.045544,0.423336,0.057030,...,0.035974,-0.044032,0.490040,0.287544,-0.584386,0.145022,0.071476,0.225753,-0.012832,-0.152288


In [4]:
metadata_columns = [x for x in df.columns if "Metadata" in x]
scDINO_columns = [x for x in df.columns if "scDINO" in x]
CP_columns = df.drop(columns=metadata_columns + scDINO_columns).columns
CP_scDINO_columns = df.drop(metadata_columns, axis=1).columns

feature_set_dict = {
    "scDINO": scDINO_columns,
    "CP": CP_columns,
    "CP_scDINO": CP_scDINO_columns,
}

In [5]:
df["Metadata_Time"] = df["Metadata_Time"].astype(float)
timepoints = df["Metadata_Time"].unique().tolist()
timepoints.sort()
shuffle_options = [False, True]

In [6]:
# This loops through the CP, scDINO, and CP_scDINO feature sets and calculates the mean average precision for each timepoint


for feature_set_name, feature_set in feature_set_dict.items():
    feature_subset_df = pd.concat([df[feature_set], df[metadata_columns]], axis=1)

    list_of_maps = []
    # calculate the mean average precision for each timepoint
    # and each shuffle option
    for timepoint in timepoints:
        time_subset_df = feature_subset_df[
            feature_subset_df["Metadata_Time"] == timepoint
        ]

        for shuffle in shuffle_options:
            if shuffle == True:
                random.seed(0)
                # permutate the data
                for col in time_subset_df.columns:
                    if col != "Metadata_dose":
                        time_subset_df.loc[:, col] = np.random.permutation(
                            time_subset_df[col]
                        )

            reference_col = "Metadata_reference_index"
            df_activity = assign_reference_index(
                time_subset_df,
                "Metadata_dose == '0.0'",
                reference_col=reference_col,
                default_value=-1,
            )
            pos_sameby = ["Metadata_dose", reference_col]
            pos_diffby = []
            neg_sameby = []
            neg_diffby = ["Metadata_dose", reference_col]
            metadata = df_activity.filter(regex="Metadata")
            profiles = df_activity.filter(regex="^(?!Metadata)")
            # drop the nans from the profiles
            profiles = profiles.dropna(axis=0)
            metadata = metadata.loc[profiles.index]
            profiles = profiles.values
            # suppress the warning
            activity_ap = map.average_precision(
                metadata, profiles, pos_sameby, pos_diffby, neg_sameby, neg_diffby
            )

            activity_ap = activity_ap.query("Metadata_dose != '0.0'")
            activity_map = map.mean_average_precision(
                activity_ap, pos_sameby, null_size=1000000, threshold=0.05, seed=0
            )
            activity_map.reset_index(inplace=True)
            activity_map.insert(
                0,
                "Metadata_Time",
                timepoint,
            )
            if shuffle:
                activity_map.insert(
                    1,
                    "Shuffle",
                    "True",
                )
            else:
                activity_map.insert(
                    1,
                    "Shuffle",
                    "False",
                )
            list_of_maps.append(activity_map)

    final_mAP_df = pd.concat(list_of_maps, ignore_index=True)
    # save the final dataframe to a parquet file
    final_mAP_path = pathlib.Path(
        f"../data/mAP/mAP_scores_{feature_set_name}.parquet"
    ).resolve()
    final_mAP_path.parent.mkdir(parents=True, exist_ok=True)
    final_mAP_df.to_parquet(
        final_mAP_path,
        index=False,
    )

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]